In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

from time import time
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from functools import partial

from tqdm import tqdm_notebook as tqdm

from PIL import Image
import cv2

import h5py

from multiprocessing import Pool as ThreadPool

#Seed for reproducable results
np.random.seed(0)

In [2]:
DATA_AMOUNT=500000

In [3]:
file_path = "/home/amitp/Documents/Python/Training Orientation Detection/training_dataset_all_doric_cabled.h5"
with h5py.File(file_path,'r') as f:
    data = f['/frames'][:DATA_AMOUNT]
    #data = data[::5]
DATA_AMOUNT=len(data)

In [4]:
def RotateFrame(frame,noise=0):
    #Angle to rotate by
    a = np.random.random()*180
    #PIL to rotate images
    rotated=np.array(Image.fromarray(frame).rotate(a))
    #Added noise for testing
    noisy=np.array(rotated)+np.random.normal(0,noise,(80,80))
    #Clamp image values above 0
    noisy[noisy<0] = 0
    return noisy,a

In [5]:
angles=np.zeros(len(data))

rotation_args = {
    "noise":0
}
#Multithreaded image rotations using tqdm for progress
pool = ThreadPool(8)
i=0
for val,a in tqdm(pool.imap(partial(RotateFrame,noise=rotation_args["noise"]),data,chunksize=100), total=len(data)):
    data[i]=val
    angles[i]=a
    i+=1
pool.close()
pool.join()

In [6]:
def OrientationFromImage(frame,debug=False,blocksize=41,sub_const=-3):
    im = cv2.adaptiveThreshold(frame,255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,blocksize,sub_const)
    im = cv2.erode(im,kernel = np.ones((4,4),np.uint8),iterations = 3)
    im = cv2.dilate(im,kernel = np.ones((4,4),np.uint8),iterations = 1)
    if debug:
        return im
    _,contours,_=cv2.findContours(im,2,2)
    if(len(contours)>0):
        cnt = contours[0]
        moments = cv2.moments(cnt)
        angle = -0.5*np.arctan2(2*moments["mu11"],moments["mu20"]-moments["mu02"])
        angle =angle*180/np.pi
        if(angle<0):
            angle=180+angle
        return angle
    else:
        return 0

In [7]:
def meanError(arr):
    for i in range(0,len(arr)):
        if(abs(angles[i]-arr[i])>170):
            arr[i]=180-arr[i]
    return np.mean(abs(angles-predics))

In [8]:
#Angle prediction array
predics= np.zeros(len(data))

prediction_args = {
    "debug":False,#Only use if you want the filtered images, not the angles
    "blocksize":39,#Block size variable for adaptive thresholding
    "subtraction_constant":-3#Constant value subtracted during thresholding
}
#Multithread predictions, using tqdm for progress bar, change the number of threads based on system.
pool = ThreadPool(8)
i=0
for val in tqdm(pool.imap(partial(OrientationFromImage,
                                  debug=prediction_args["debug"],
                                  blocksize=prediction_args["blocksize"],
                                  sub_const=prediction_args["subtraction_constant"]),
                          data,chunksize=1000), total=len(data)):
    predics[i]=val
    i+=1
pool.close()
pool.join()
#Stats
mean_error = meanError(predics)
print("Average Error (deg) for test:" , mean_error,end='')


Average Error (deg) for test: 2.730664776971453